<a href="https://colab.research.google.com/github/eugene123tw/cs330-hw1/blob/master/HW1_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Downlaod Omniglot 
workspace_dir = '.'
!gdown --id 1aBacYkuigdlKExME-kgxqworbdd8Zixg --output "{workspace_dir}/omniglot_resized.zip"

Downloading...
From: https://drive.google.com/uc?id=1aBacYkuigdlKExME-kgxqworbdd8Zixg
To: /content/omniglot_resized.zip
13.0MB [00:00, 205MB/s]


In [0]:
!unzip -q omniglot_resized

In [13]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu Apr  9 13:32:03 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [10]:
import os
import random

import numpy as np
from skimage.io import imread
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

def get_images(paths, labels, nb_samples=None, shuffle=True):
  """
  Takes a set of character folders and labels and returns paths to image files
  paired with labels.
  Args:
      paths: A list of character folders
      labels: List or numpy array of same length as paths
      nb_samples: Number of images to retrieve per character
  Returns:
      List of (label, image_path) tuples
  """
  if nb_samples is not None:
    sampler = lambda x: random.sample(x, nb_samples)
  else:
    sampler = lambda x: x
  images_labels = [(i, os.path.join(path, image))
                   for i, path in zip(labels, paths)
                   for image in sampler(os.listdir(path))]
  if shuffle:
    random.shuffle(images_labels)
  return images_labels


def image_file_to_array(filename, dim_input):
  """
  Takes an image path and returns numpy array
  Args:
      filename: Image filename
      dim_input: Flattened shape of image
  Returns:
      1 channel image
  """
  image = imread(filename)
  image = image.reshape([dim_input])
  image = image.astype(np.float32) / 255.0
  image = 1.0 - image
  return image


class DataGenerator(object):
  """
  Data Generator capable of generating batches of Omniglot data.
  A "class" is considered a class of omniglot digits.
  """

  def __init__(self, num_classes, num_samples_per_class, config={}):
    """
    Args:
        num_classes: Number of classes for classification (K-way)
        num_samples_per_class: num samples to generate per class in one batch
        batch_size: size of meta batch size (e.g. number of functions)
    """
    self.num_samples_per_class = num_samples_per_class
    self.num_classes = num_classes

    data_folder = config.get('data_folder', './omniglot_resized')
    self.img_size = config.get('img_size', (28, 28))

    self.dim_input = np.prod(self.img_size)
    self.dim_output = self.num_classes

    character_folders = [os.path.join(data_folder, family, character)
                         for family in os.listdir(data_folder)
                         if os.path.isdir(os.path.join(data_folder, family))
                         for character in os.listdir(os.path.join(data_folder, family))
                         if os.path.isdir(os.path.join(data_folder, family, character))]

    random.seed(1)
    random.shuffle(character_folders)
    num_val = 100
    num_train = 1100
    self.metatrain_character_folders = character_folders[: num_train]
    self.metaval_character_folders = character_folders[
                                     num_train:num_train + num_val]
    self.metatest_character_folders = character_folders[
                                      num_train + num_val:]

  def sample_batch(self, batch_type, batch_size):
    """
    Samples a batch for training, validation, or testing
    Args:
        batch_type: train/val/test
    Returns:
        A a tuple of (1) Image batch and (2) Label batch where
        image batch has shape [B, K, N, 784] and label batch has shape [B, K, N, N]
        where B is batch size, K is number of samples per class, N is number of classes
    """
    if batch_type == "train":
      folders = self.metatrain_character_folders
    elif batch_type == "val":
      folders = self.metaval_character_folders
    else:
      folders = self.metatest_character_folders

    #############################
    #### YOUR CODE GOES HERE ####
    # Initialise array for storage
    all_image_batches = np.zeros(
      (batch_size, self.num_samples_per_class, self.num_classes, 784)
    )

    all_label_batches = np.zeros(
      (batch_size, self.num_samples_per_class, self.num_classes, self.num_classes)
    )

    for b in range(batch_size):  # sample mini batch of tasks
      sampled_paths = np.random.choice(folders, self.num_classes)
      images_labels = get_images(
        sampled_paths, np.eye(self.num_classes), self.num_samples_per_class, False)

      for k in range(self.num_samples_per_class):
        s = slice(k, len(images_labels), self.num_samples_per_class)
        kth_images_labels = images_labels[s]
        np.random.shuffle(kth_images_labels)
        for n, (y_vector, fname) in enumerate(kth_images_labels):
          all_image_batches[b, k, n, :] = image_file_to_array(fname, 784)
          all_label_batches[b, k, n, :] = y_vector
    #############################

    return all_image_batches, all_label_batches

Instructions for updating:
non-resource variables are not supported in the long term


In [0]:
def loss_function(preds, real_labels):
  """
  Computes MANN loss
  Args:
      preds: [B, K+1, N, N] network output
      labels: [B, K+1, N, N] labels
  Returns:
      scalar loss
  """
  #############################
  #### YOUR CODE GOES HERE ####
  loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
  b, k, n, n = real_labels.get_shape().as_list()
  labels = tf.reshape(real_labels, (-1, k * n, n))
  return loss(labels[:, -1:], preds[:, -1:])
  #############################


class MANN(tf.keras.Model):

  def __init__(self, num_classes, samples_per_class):
    super(MANN, self).__init__()
    self.num_classes = num_classes
    self.samples_per_class = samples_per_class
    self.layer1 = tf.keras.layers.LSTM(128, return_sequences=True)
    self.layer2 = tf.keras.layers.LSTM(num_classes, return_sequences=True)

  def call(self, input_images, fake_labels):
    """
    MANN
    Args:
        input_images: [B, K+1, N, 11025] flattened images
        labels: [B, K+1, N, N] ground truth labels
    Returns:
        [B, K+1, N, N] predictions
    """
    #############################
    #### YOUR CODE GOES HERE ####
    K = self.samples_per_class
    N = self.num_classes
    input_images = tf.reshape(input_images, (-1, K * N, 784))
    fake_labels = tf.reshape(fake_labels, (-1, K * N, N))
    input_tensor = tf.concat([input_images, fake_labels], axis=2)
    out = self.layer1(input_tensor)
    out = self.layer2(out)
    #############################
    return out

# Start Training

In [0]:
import argparse

parser = argparse.ArgumentParser(description='Process some flags.')

parser.add_argument('--num_classes', type=int, default=5, help='number of classes used in classification (e.g. 5-way classification).')

parser.add_argument('--num_samples', type=int, default=1, help='number of examples used for inner gradient update (K for K-shot learning).')

parser.add_argument('--meta_batch_size', type=int, default=16, help='Number of N-way classification tasks per batch')

parser.add_argument('--data_root', type=str, default= 'omniglot_resized', help='data folder root')

FLAGS = parser.parse_args(args=[])

In [0]:
ims = tf.placeholder(tf.float32, shape=(None, FLAGS.num_samples + 1, FLAGS.num_classes, 784))

fake_labels = tf.placeholder(tf.float32, shape=(None, FLAGS.num_samples + 1, FLAGS.num_classes, FLAGS.num_classes))

real_labels = tf.placeholder(tf.float32, shape=(None, FLAGS.num_samples + 1, FLAGS.num_classes, FLAGS.num_classes))

data_generator = DataGenerator(
  FLAGS.num_classes,
  FLAGS.num_samples + 1,
  {'data_folder': FLAGS.data_root}
)

o = MANN(FLAGS.num_classes, FLAGS.num_samples + 1)
out = o(ims, fake_labels)

loss = loss_function(out, real_labels)
optim = tf.train.AdamOptimizer(0.001)
optimizer_step = optim.minimize(loss)

with tf.Session() as sess:
  sess.run(tf.local_variables_initializer())
  sess.run(tf.global_variables_initializer())

  for step in range(50000):
    i, l = data_generator.sample_batch('train', FLAGS.meta_batch_size)
    l_fake = l.copy()
    l_fake[:, -1:] = 0
    feed = {
      ims: i.astype(np.float32),
      fake_labels: l_fake.astype(np.float32),
      real_labels: l.astype(np.float32),
    }
    _, ls = sess.run([optimizer_step, loss], feed)

    if step % 100 == 0:
      print("*" * 5 + "Iter " + str(step) + "*" * 5)
      i, l = data_generator.sample_batch('test', 100)
      l_fake = l.copy()
      l_fake[:, -1:] = 0
      feed = {
        ims: i.astype(np.float32),
        fake_labels: l_fake.astype(np.float32),
        real_labels: l.astype(np.float32),
      }
      pred, tls = sess.run([out, loss], feed)
      print("Train Loss:", ls, "Test Loss:", tls)
      pred = pred.reshape(
        -1, FLAGS.num_samples + 1,
        FLAGS.num_classes, FLAGS.num_classes)
      pred = pred[:, -1, :, :].argmax(2)
      l = l[:, -1, :, :].argmax(2)
      print("Test Accuracy", (1.0 * (pred == l)).mean())


*****Iter 0*****
Train Loss: 1.587503 Test Loss: 1.6143907
Test Accuracy 0.216
*****Iter 100*****
Train Loss: 1.6186326 Test Loss: 1.6091939
Test Accuracy 0.198
*****Iter 200*****
Train Loss: 1.6094382 Test Loss: 1.6094393
Test Accuracy 0.202
*****Iter 300*****
Train Loss: 1.609438 Test Loss: 1.6094381
Test Accuracy 0.2
*****Iter 400*****
Train Loss: 1.6094383 Test Loss: 1.6094377
Test Accuracy 0.204
*****Iter 500*****
Train Loss: 1.6094382 Test Loss: 1.6094382
Test Accuracy 0.202
*****Iter 600*****
Train Loss: 1.6094382 Test Loss: 1.6094383
Test Accuracy 0.198
*****Iter 700*****
Train Loss: 1.609438 Test Loss: 1.6094377
Test Accuracy 0.2
*****Iter 800*****
Train Loss: 1.6094381 Test Loss: 1.6094377
Test Accuracy 0.198
*****Iter 900*****
Train Loss: 1.6094382 Test Loss: 1.6094378
Test Accuracy 0.198
*****Iter 1000*****
Train Loss: 1.609438 Test Loss: 1.6094382
Test Accuracy 0.196
*****Iter 1100*****
Train Loss: 1.6094378 Test Loss: 1.6094378
Test Accuracy 0.2
*****Iter 1200*****
Train 

In [0]:
print(FLAGS.data_root)